<a href="https://colab.research.google.com/github/Chryron/CSC2516_NN-DL/blob/main/CSC2516_Homework_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Backpropagation for bias vectors (1.5 points)

In class, we discussed a multilayer perceptron (neural network) whose layers were all "dense", i.e. the output $a^m \in \mathbb{R}^{N^m}$ of the $m$th layer is computed as
\begin{align*}
z^m &= W^m a^{m - 1} + b^m \\
a^m &= \sigma^m(z^m)
\end{align*}
where $W^m \in \mathbb{R}^{N^m \times N^{m - 1}}$ is the weight matrix, $b^m \in \mathbb{R}^{N^m}$ is the bias vector, and $\sigma^m$ is the nonlinearity. We showed that
$$\frac{\partial C}{\partial W^m} = \frac{\partial C}{\partial z^m} a^{m - 1 \top}$$
Show that
$$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$
Hint: The derivation is almost the same as for $W$.

$$
\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}\frac{\partial z^m}{\partial b^m}
$$
Considering elementwise partials,
$$
\begin{align*}
\frac{\partial C}{\partial b^m_k} &= \frac{\partial C}{\partial z^m_k}\frac{\partial z^m_k}{\partial b^m_k} \\ 
z^m_k &= \sum_{j=1}^{N^m} (W^m_{kj}a^m_j) + b^m_k \\ 
\text{so, }\frac{\partial z^m_k}{\partial b^m_k} &= 1 \\ 
\frac{\partial C}{\partial b^m_k} &= \frac{\partial C}{\partial z^m_k} \cdot 1 \\ 
\end{align*}
$$
Thus, in vector form: 
$$\frac{\partial C}{\partial b^m} = \frac{\partial C}{\partial z^m}$$

# 2. MLP from scratch (3.5 points)

Using numpy only, implement backward pass or a sigmoid MLP. Specifically, you will need to implement this functionality in the `train` function in the `SigmoidMLP` class below. You should write numpy code to populate the two lists `weight_gradients` and `bias_gradient`, where each entry in each list corresponds to the gradient for a weight matrix or bias vector for each layer. Then, when you run the code cell at the bottom of this notebook, the trained MLP should output (approximately) 0, 1, 1, 0, having learned the [XOR function](https://en.wikipedia.org/wiki/Exclusive_or). Please us a binary cross-entropy loss, i.e.
$$C(a^L, y) = (y - 1)\log(1 - a^L) - y\log(a^L)$$

*Note 1*: All layers in your model, including the last layer, will use the sigmoid nonlinearity. Remember that
$$
\frac{\partial}{\partial x}\mathrm{sigmoid}(x) = \mathrm{sigmoid}(x)(1 - \mathrm{sigmoid}(x))$$

*Note 2*: As we mentioned in class,
$$
\frac{\partial C}{\partial z^L} = a^L - y
$$

In [32]:
import numpy as np

def sigmoid(X):
    return 1/(1 + np.exp(-X))

class Layer:
    def __init__(self, inputs, outputs):
        # Initialize weight matrix and bias vector
        # Getting the initialization right can be tricky, but for this problem
        # simply drawing from a standard normal distribution should work.
        self.weights = np.random.randn(outputs, inputs)
        self.biases = np.random.randn(outputs, 1)
    def __call__(self, X):
        # Compute \sigmoid(Wx + b)
        return 1/(1 + np.exp(-(self.weights.dot(X) + self.biases)))

class SigmoidMLP:

    def __init__(self, layer_widths):
        self.layers = []
        for inputs, outputs in zip(layer_widths[:-1], layer_widths[1:]):
            self.layers.append(Layer(inputs, outputs))

    def train(self, inputs, targets, learning_rate):
        # Forward pass - compute each layer's output and store it for later use
        layer_outputs = [inputs]
        for layer in self.layers:
            layer_outputs.append(layer(layer_outputs[-1]))

        # Implement backward pass to populate weight_gradients and bias_gradients
        # lists here
        weight_gradients = []
        bias_gradients = []
        
        layer_outputs.reverse()
        i = 0
        for current_output in layer_outputs:
            if i == 0:
                dc_dz_m =  current_output - targets
                bias_gradients.append(np.mean(dc_dz_m, axis=1, keepdims=True)) # averaging because doing batch training with multiple examples instead of just 1
                weight_gradients.append(dc_dz_m @ layer_outputs[i+1].transpose())
            else:
                forward_layer = self.layers[-i]
                da_dz = current_output*(1-current_output)
                dc_dz_m = (forward_layer.weights.transpose() @ dc_dz_m) * da_dz
                bias_gradients.append(np.mean(dc_dz_m, axis=1, keepdims=True)) # averaging because doing batch training with multiple examples instead of just 1
                weight_gradients.append(dc_dz_m @ layer_outputs[i+1].transpose())
            
            if i == len(layer_outputs) - 1 - 1: # index of output of first layer
                break
            i+=1
        
        weight_gradients.reverse()
        bias_gradients.reverse()

        # Perform gradient descent by applying updates
        for weight_gradient, bias_gradient, layer in zip(weight_gradients, bias_gradients, self.layers):
            layer.weights -= weight_gradient*learning_rate
            layer.biases -= bias_gradient*learning_rate

    def __call__(self, inputs):
        a = inputs
        for layer in self.layers:
            a = layer(a)
        return a

def train_mlp(n_iterations, learning_rate):
    mlp = SigmoidMLP([2, 2, 1])
    inputs = np.array([[0, 1, 0, 1],
                       [0, 0, 1, 1]])
    targets = np.array([[0, 1, 1, 0]])
    for _ in range(int(1e3)):
        mlp.train(inputs, targets, learning_rate)
    return mlp

In [33]:
# You may need to change the n_iterations and learning_rate values
# but these worked for me
mlp = train_mlp(10000, 1.)
# The following calls should result in (approximately) 0, 1, 1, 0
# If the outputs are somewhat close, your training has succeeded!
print(mlp(np.array([0, 0]).reshape(-1, 1)))
print(mlp(np.array([0, 1]).reshape(-1, 1)))
print(mlp(np.array([1, 0]).reshape(-1, 1)))
print(mlp(np.array([1, 1]).reshape(-1, 1)))

[[0.00284373]]
[[0.9976219]]
[[0.9933704]]
[[0.00182836]]
